In [2]:
import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.feature_extraction.text import CountVectorizer
import random
from sklearn import preprocessing
import lightgbm as lgb
import gc
from scipy.stats import skew, boxcox
from bayes_opt import BayesianOptimization
from scipy import sparse
from sklearn.metrics import log_loss
from datetime import datetime
from scipy.stats.mstats import gmean

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

seed = 2017

# Load Data

In [3]:
data_path = "../input/"
train_X = pd.read_csv(data_path + 'train_BM_MB_add03052240.csv')
test_X = pd.read_csv(data_path + 'test_BM_MB_add03052240.csv')
train_y = np.ravel(pd.read_csv(data_path + 'labels_BrandenMurray.csv'))
ntrain = train_X.shape[0]
sub_id = test_X.listing_id.values
# all_features = features_to_use + desc_sparse_cols + feat_sparse_cols
print train_X.shape, test_X.shape, train_y.shape

(49352, 322) (74659, 322) (49352L,)


In [4]:
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, train_size=.80, random_state=1234)
print X_train.shape
print X_val.shape

import sys  
stdi,stdo,stde=sys.stdin,sys.stdout,sys.stderr
reload(sys)  
sys.stdin,sys.stdout,sys.stderr=stdi,stdo,stde
sys.setdefaultencoding('utf8')

(39481, 322)
(9871, 322)


In [14]:
# clf = lgb.LGBMClassifier()
# clf.set_params(learning_rate = 0.1)
# clf.set_params(subsample_freq = 1)
# clf.set_params(objective = 'multiclass')
# clf.set_params(n_estimators = 100000)
        
# clf = clf.fit(X_train, y_train,
#               eval_set = [(X_val,y_val)],
#               eval_metric = 'multi_logloss',
#               early_stopping_rounds = 50,
#               verbose = 25)

In [15]:
# pred_y = clf.predict_proba(test_X, num_iteration = clf.best_iteration)

In [19]:
# now = datetime.now()
# sub_name = '../output/sub_LightGBM_BM_0322_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'

# out_df = pd.DataFrame(pred_y[:,:3])
# out_df.columns = ["low", "medium", "high"]
# out_df["listing_id"] = sub_id
# out_df.to_csv(sub_name, index=False)

# Tune LightGBM

In [23]:
clf = lgb.LGBMClassifier()
clf.set_params(learning_rate = 0.1)
clf.set_params(subsample_freq = 1)
clf.set_params(objective = 'multiclass')
clf.set_params(n_estimators = 100000)
clf.set_params(boosting_type = 'dart')
clf.set_params(seed = seed)
tmp  = 1000

In [24]:
for x in [8,15,31,63,127,255]:
    clf.set_params(num_leaves = x)
    clf = clf.fit(X_train, y_train,
                  eval_set = [(X_val,y_val)],
                  eval_metric = 'multi_logloss',
                  early_stopping_rounds = 50,
                  verbose = False)
    if tmp > clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]:
        num_leaves = x
        tmp = clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]


    print x, ' \t', clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1], clf.best_iteration

8  	0.541015795481 1394
15  	0.535520643335 1116
31  	0.532809905024 778
63  	0.533565449038 625
127  	0.536832255243 325
255  	0.541986381983 245


In [25]:
clf.set_params(num_leaves = num_leaves)

LGBMClassifier(boosting_type='dart', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=5,
        min_split_gain=0, n_estimators=100000, nthread=-1, num_leaves=31,
        objective='multiclass', reg_alpha=0, reg_lambda=0,
        scale_pos_weight=1, seed=2017, sigmoid=1.0, silent=True,
        skip_drop=0.5, subsample=1, subsample_for_bin=50000,
        subsample_freq=1, uniform_drop=False, xgboost_dart_mode=False)

In [26]:
min_child_samples = 10

for x in [20, 30, 50, 70, 80,90,100,110,120,150,170,200,230,260]:
    clf.set_params(min_child_samples = x)
    clf = clf.fit(X_train, y_train,
                  eval_set = [(X_val,y_val)],
                  eval_metric = 'multi_logloss',
                  early_stopping_rounds = 50,
                  verbose = False)
    if tmp > clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]:
        min_child_samples = x
        tmp = clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]


    print x, ' \t', clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1], clf.best_iteration

20  	0.539272604499 407
30  	0.543087827387 285
50  	0.533818564892 750
70  	0.533334507269 810
80  	0.533467385285 668
90  	0.534999595688 645
100  	0.535044396376 549
110  	0.530928410377 880
120  	0.532047199131 881
150  	0.532855052432 807
170  	0.533691244069 781
200  	0.53325817989 759
230  	0.532611706884 890
260  	0.533057418827 915


In [27]:
for x in [300,350,400,450,500]:
    clf.set_params(min_child_samples = x)
    clf = clf.fit(X_train, y_train,
                  eval_set = [(X_val,y_val)],
                  eval_metric = 'multi_logloss',
                  early_stopping_rounds = 50,
                  verbose = False)
    if tmp > clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]:
        min_child_samples = x
        tmp = clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]


    print x, ' \t', clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1], clf.best_iteration

300  	0.532911772246 954
350  	0.534287413942 967
400  	0.534026193565 911
450  	0.534118220251 809
500  	0.533101353311 938


In [28]:
clf

LGBMClassifier(boosting_type='dart', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=500, min_child_weight=5,
        min_split_gain=0, n_estimators=100000, nthread=-1, num_leaves=31,
        objective='multiclass', reg_alpha=0, reg_lambda=0,
        scale_pos_weight=1, seed=2017, sigmoid=1.0, silent=True,
        skip_drop=0.5, subsample=1, subsample_for_bin=50000,
        subsample_freq=1, uniform_drop=False, xgboost_dart_mode=False)

In [29]:
print min_child_samples
clf.set_params(min_child_samples = min_child_samples)


110


LGBMClassifier(boosting_type='dart', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=110, min_child_weight=5,
        min_split_gain=0, n_estimators=100000, nthread=-1, num_leaves=31,
        objective='multiclass', reg_alpha=0, reg_lambda=0,
        scale_pos_weight=1, seed=2017, sigmoid=1.0, silent=True,
        skip_drop=0.5, subsample=1, subsample_for_bin=50000,
        subsample_freq=1, uniform_drop=False, xgboost_dart_mode=False)

In [30]:
colsample_bytree = 1
for x in [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    clf.set_params(colsample_bytree = x)
    clf = clf.fit(X_train, y_train,
                  eval_set = [(X_val,y_val)],
                  eval_metric = 'multi_logloss',
                  early_stopping_rounds = 50,
                  verbose = False)
    if tmp > clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]:
        colsample_bytree = x
        tmp = clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]


    print x, ' \t', clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1], clf.best_iteration

0.2  	0.530523614238 1319
0.3  	0.528740074153 1151
0.4  	0.52847335583 1200
0.5  	0.529425519615 1170
0.6  	0.531773805656 801
0.7  	0.537678359227 407
0.8  	0.532697221123 771
0.9  	0.533260395329 806


In [31]:
print colsample_bytree

clf.set_params(colsample_bytree = colsample_bytree)

0.4


LGBMClassifier(boosting_type='dart', colsample_bytree=0.4, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=110, min_child_weight=5,
        min_split_gain=0, n_estimators=100000, nthread=-1, num_leaves=31,
        objective='multiclass', reg_alpha=0, reg_lambda=0,
        scale_pos_weight=1, seed=2017, sigmoid=1.0, silent=True,
        skip_drop=0.5, subsample=1, subsample_for_bin=50000,
        subsample_freq=1, uniform_drop=False, xgboost_dart_mode=False)

In [32]:
subsample = 1.0
for x in [0.5,0.6,0.7,0.8,0.9]:
    clf.set_params(subsample = x)
    clf = clf.fit(X_train, y_train,
                  eval_set = [(X_val,y_val)],
                  eval_metric = 'multi_logloss',
                  early_stopping_rounds = 50,
                  verbose = False)
    if tmp > clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]:
        subsample = x
        tmp = clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]


    print x, ' \t', clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1], clf.best_iteration

0.5  	0.534293883688 809
0.6  	0.534663564215 806
0.7  	0.529391220803 1323
0.8  	0.532074542962 810
0.9  	0.532550988969 755


In [33]:
print subsample
clf.set_params(subsample = subsample)

1.0


LGBMClassifier(boosting_type='dart', colsample_bytree=0.4, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=110, min_child_weight=5,
        min_split_gain=0, n_estimators=100000, nthread=-1, num_leaves=31,
        objective='multiclass', reg_alpha=0, reg_lambda=0,
        scale_pos_weight=1, seed=2017, sigmoid=1.0, silent=True,
        skip_drop=0.5, subsample=1.0, subsample_for_bin=50000,
        subsample_freq=1, uniform_drop=False, xgboost_dart_mode=False)

In [34]:
max_bin = 255

for x in [15,31,63, 127, 511, 1023, 2047]: #[200,300,400]:#
    clf.set_params(max_bin = x)
    clf = clf.fit(X_train, y_train,
                  eval_set = [(X_val,y_val)],
                  eval_metric = 'multi_logloss',
                  early_stopping_rounds = 50,
                  verbose = False)
    if tmp > clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]:
        max_bin = x
        tmp = clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]


    print x, ' \t', clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1], clf.best_iteration

15  	0.536688857754 1049
31  	0.534468185301 935
63  	0.530532367141 1155
127  	0.540523135824 407
511  	0.53000503041 952
1023  	0.539521507992 407
2047  	0.53093937159 957


In [35]:
for x in [210,240,270,300,350,400,450]:
    clf.set_params(max_bin = x)
    clf = clf.fit(X_train, y_train,
                  eval_set = [(X_val,y_val)],
                  eval_metric = 'multi_logloss',
                  early_stopping_rounds = 50,
                  verbose = False)
    if tmp > clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]:
        max_bin = x
        tmp = clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]


    print x, ' \t', clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1], clf.best_iteration

210  	0.539239337795 407
240  	0.529590411308 898
270  	0.528406563042 1230
300  	0.529405419661 1051
350  	0.532100747004 810
400  	0.531729001523 771
450  	0.539675397699 410


In [36]:
print max_bin
clf.set_params(max_bin = max_bin)

270


LGBMClassifier(boosting_type='dart', colsample_bytree=0.4, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=270, max_depth=-1,
        max_drop=50, min_child_samples=110, min_child_weight=5,
        min_split_gain=0, n_estimators=100000, nthread=-1, num_leaves=31,
        objective='multiclass', reg_alpha=0, reg_lambda=0,
        scale_pos_weight=1, seed=2017, sigmoid=1.0, silent=True,
        skip_drop=0.5, subsample=1.0, subsample_for_bin=50000,
        subsample_freq=1, uniform_drop=False, xgboost_dart_mode=False)

In [37]:
drop_rate = 0.1
# clf.set_params(max_drop = 50)

for x in [0.05,0.15,0.2, 0.25, 0.3]: 
    clf.set_params(drop_rate = x)
    clf = clf.fit(X_train, y_train,
                  eval_set = [(X_val,y_val)],
                  eval_metric = 'multi_logloss',
                  early_stopping_rounds = 50,
                  verbose = False)
    if tmp > clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]:
        drop_rate = x
        tmp = clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]


    print x, ' \t', clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1], clf.best_iteration

0.05  	0.530407385003 910
0.15  	0.530345892393 1114
0.2  	0.530500969161 957
0.25  	0.530247231038 1051
0.3  	0.52912889179 1116


In [42]:
clf.set_params(max_drop = 50)
clf.set_params(drop_rate = 0.1)
skip_drop = 0.5
for x in [0.3,0.4,0.6,0.7]: 
    clf.set_params(skip_drop = x)
    clf = clf.fit(X_train, y_train,
                  eval_set = [(X_val,y_val)],
                  eval_metric = 'multi_logloss',
                  early_stopping_rounds = 50,
                  verbose = False)
    if tmp > clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]:
        skip_drop = x
        tmp = clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]


    print x, ' \t', clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1], clf.best_iteration

0.3  	0.582516152841 221
0.4  	0.549728883413 372
0.6  	0.528374802311 864
0.7  	0.529720291491 629


In [43]:
for x in [0.8,0.9,1.0]: 
    clf.set_params(skip_drop = x)
    clf = clf.fit(X_train, y_train,
                  eval_set = [(X_val,y_val)],
                  eval_metric = 'multi_logloss',
                  early_stopping_rounds = 50,
                  verbose = False)
    if tmp > clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]:
        skip_drop = x
        tmp = clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1]


    print x, ' \t', clf.evals_result.values()[0]['multi_logloss'][clf.best_iteration -1], clf.best_iteration

0.8  	0.528899728631 557
0.9  	0.53059101276 504
1.0  	0.53231000987 309


In [44]:
clf

LGBMClassifier(boosting_type='dart', colsample_bytree=0.4, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=270, max_depth=-1,
        max_drop=50, min_child_samples=110, min_child_weight=5,
        min_split_gain=0, n_estimators=100000, nthread=-1, num_leaves=31,
        objective='multiclass', reg_alpha=0, reg_lambda=0,
        scale_pos_weight=1, seed=2017, sigmoid=1.0, silent=True,
        skip_drop=1.0, subsample=1.0, subsample_for_bin=50000,
        subsample_freq=1, uniform_drop=False, xgboost_dart_mode=False)

In [ ]:
def lgbm_cv(max_bin, num_leaves, min_child_samples, colsample_bytree, subsample,
            drop_rate, skip_drop, learning_rate=0.1):
    skf = KFold(n_splits=5,random_state=seed)
    scores=[]
    for i, (train, val) in enumerate(skf.split(train_X)):
        est=lgb.LGBMClassifier(boosting_type = 'dart',
                               seed = seed,
                               learning_rate=0.1,
                               max_bin=int(max_bin),
                               num_leaves=int(num_leaves),
                               min_child_samples=int(min_child_samples),
                               colsample_bytree=colsample_bytree,
                               subsample=subsample,
                               subsample_freq = 1,
                               drop_rate = drop_rate,
                               skip_drop = skip_drop
                              )
 
        train_x_fold = train_X.iloc[train]
        train_y_fold = train_y[train]
        val_x_fold = train_X.iloc[val]
        val_y_fold = train_y[val]
        est.set_params(n_estimators=100000)
        est.fit(train_x_fold,
                train_y_fold,
                eval_set=[(val_x_fold, val_y_fold)],
                eval_metric='multi_logloss',
                early_stopping_rounds=50,
                verbose = False
               )
        val_y_predict_fold = est.predict_proba(val_x_fold)
        score = log_loss(val_y_fold, val_y_predict_fold)
        scores.append(score)
    return -np.mean(scores)


lgbm_BO = BayesianOptimization(lgbm_cv,
                               {
        'max_bin': (63,511),
        'num_leaves': (15,127),
        'min_child_samples' :(100,350),
        'colsample_bytree': (0.2,0.7),
        'subsample' : (0.7,1),
        'drop_rate' : (0.05,0.4),
        'skip_drop' : (0.4,0.9)
    })

lgbm_BO.maximize(init_points=10, n_iter=40)

In [50]:
gbm_bo_scores = pd.DataFrame([[s[0]['num_leaves'],
                               s[0]['min_child_samples'],
                               s[0]['max_bin'],
                               s[0]['colsample_bytree'],
                               s[0]['subsample'],
                               s[0]['drop_rate'],
                               s[0]['skip_drop'],
                               s[1]] for s in zip(lgbm_BO.res['all']['params'],lgbm_BO.res['all']['values'])],
                            columns = ['num_leaves',
                                       'min_child_samples',
                                       'max_bin',
                                       'colsample_bytree',
                                       'subsample',
                                       'drop_rate',
                                       'skip_drop',
                                       'score'])
gbm_bo_scores=gbm_bo_scores.sort_values('score',ascending=False)
gbm_bo_scores.head(10)

,num_leaves,min_child_samples,max_bin,colsample_bytree,subsample,drop_rate,skip_drop,score
31,24.369054,105.999784,347.890343,0.296539,0.958292,0.296937,0.761489,-0.527973
25,36.551311,103.138433,372.848850,0.341755,0.897879,0.351620,0.820402,-0.528236
16,24.835812,127.011433,379.260023,0.692687,0.983991,0.318513,0.779600,-0.528337
7,53.980917,128.971711,200.086203,0.371401,0.868250,0.166791,0.607333,-0.528482
1,20.459668,127.677774,224.976864,0.633188,0.953674,0.058562,0.855635,-0.528548
5,36.393594,107.774077,162.760818,0.538180,0.962790,0.294802,0.831861,-0.528939
29,65.263138,103.264768,68.005549,0.441635,0.977634,0.367447,0.688544,-0.529395
11,23.274251,106.841420,257.351517,0.437205,0.721438,0.339741,0.878603,-0.529592
9,22.450374,111.507409,70.716295,0.356903,0.860929,0.275919,0.581766,-0.529795
18,62.818540,100.065105,305.343533,0.459058,0.828485,0.167969,0.679024,-0.529982


In [52]:
def lgbm_blend(estimators, train_x, train_y, test_x, fold, early_stopping_rounds=50):
    N_params = len(estimators)
    print ("Blend %d estimators for %d folds" % (N_params, fold))
    skf = KFold(n_splits=fold,random_state=seed)
    N_class = len(set(train_y))
    
    train_blend_x = np.zeros((train_x.shape[0], N_class*N_params))
    test_blend_x_mean = np.zeros((test_x.shape[0], N_class*N_params))
    test_blend_x_gmean = np.zeros((test_x.shape[0], N_class*N_params))
    scores = np.zeros((fold,N_params))
    best_rounds = np.zeros((fold, N_params))    

    
    for j, est in enumerate(estimators):
        est.set_params(learning_rate = 0.01)
        est.set_params(subsample_freq = 1)
        est.set_params(objective = 'multiclass')
        est.set_params(n_estimators = 1000000)
        est.set_params(boosting_type = 'dart')
        est.set_params(seed = seed)
        
        print ("Model %d: %s" %(j+1, est)) 

        
        test_blend_x_j = np.zeros((test_x.shape[0], N_class*fold))
        for i, (train_index, val_index) in enumerate(skf.split(train_x)):
            print ("Model %d fold %d" %(j+1,i+1))
            fold_start = time.time() 
            train_x_fold = train_x.iloc[train_index]
            train_y_fold = train_y[train_index]
            val_x_fold = train_x.iloc[val_index]
            val_y_fold = train_y[val_index]
            
            est.fit(train_x_fold, train_y_fold,
                   eval_set = [(val_x_fold,val_y_fold)],
                   eval_metric = 'multi_logloss',
                   early_stopping_rounds = early_stopping_rounds,
                   verbose = False)
            
            best_round=est.best_iteration
            best_rounds[i,j]=best_round
            print ("best round %d" % (best_round))
            
            val_y_predict_fold = est.predict_proba(val_x_fold,num_iteration = best_round)
            score = log_loss(val_y_fold, val_y_predict_fold)
            print ("Score: ", score)
            scores[i,j]=score   
            train_blend_x[val_index, (j*N_class):(j+1)*N_class] = val_y_predict_fold
            test_blend_x_j[:,(i*N_class):(i+1)*N_class] = est.predict_proba(test_x,num_iteration=best_round)
            
            print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))            
            
           
        test_blend_x_mean[:,(j*N_class):(j+1)*N_class] = \
                np.stack([test_blend_x_j[:,range(0,N_class*fold,N_class)].mean(1),
                          test_blend_x_j[:,range(1,N_class*fold,N_class)].mean(1),
                          test_blend_x_j[:,range(2,N_class*fold,N_class)].mean(1)]).T
        
        test_blend_x_gmean[:,(j*N_class):(j+1)*N_class] = \
                np.stack([gmean(test_blend_x_j[:,range(0,N_class*fold,N_class)], axis=1),
                          gmean(test_blend_x_j[:,range(1,N_class*fold,N_class)], axis=1),
                          gmean(test_blend_x_j[:,range(2,N_class*fold,N_class)], axis=1)]).T
            
        print ("Score for model %d is %f" % (j+1,np.mean(scores[:,j])))
    print ("Score for blended models is %f" % (np.mean(scores)))
    return (train_blend_x, test_blend_x_mean, test_blend_x_gmean, scores,best_rounds)

In [53]:
est       = [lgb.LGBMClassifier(num_leaves = 24,
                                min_child_samples = 105,
                                colsample_bytree = 0.296539,
                                subsample = 0.958292,
                                max_bin = 347,
                                drop_rate = 0.296937,
                                skip_drop = 0.761489),
             lgb.LGBMClassifier(num_leaves = 36,
                                min_child_samples = 103,
                                colsample_bytree = 0.341755,
                                subsample = 0.897879,
                                max_bin = 372,
                                drop_rate = 0.351620,
                                skip_drop = 0.820402),
             lgb.LGBMClassifier(num_leaves = 24,
                                min_child_samples = 127,
                                colsample_bytree = 0.692687,
                                subsample = 0.983991,
                                max_bin = 379,
                                drop_rate = 0.318513,
                                skip_drop = 0.779600),
             lgb.LGBMClassifier(num_leaves = 53,
                                min_child_samples = 128,
                                colsample_bytree = 0.371401,
                                subsample = 0.868250,
                                max_bin = 200,
                                drop_rate = 0.166791,
                                skip_drop = 0.607333),
             lgb.LGBMClassifier(num_leaves = 20,
                                min_child_samples = 127,
                                colsample_bytree = 0.633188,
                                subsample = 0.953674,
                                max_bin = 224,
                                drop_rate = 0.058562,
                                skip_drop = 0.855635)]
#  		num_leaves 	min_child_samples 	max_bin 	colsample_bytree 	subsample 	drop_rate 	skip_drop 	score
# 31 	24.369054 	105.999784 			347.890343 	0.296539 			0.958292 	0.296937 	0.761489 	-0.527973
# 25 	36.551311 	103.138433 			372.848850 	0.341755 			0.897879 	0.351620 	0.820402 	-0.528236
# 16 	24.835812 	127.011433 			379.260023 	0.692687 			0.983991 	0.318513 	0.779600 	-0.528337
# 7 	53.980917 	128.971711 			200.086203 	0.371401 			0.868250 	0.166791 	0.607333 	-0.528482
# 1 	20.459668 	127.677774 			224.976864 	0.633188 			0.953674 	0.058562 	0.855635 	-0.528548

(train_blend_x_gbm,
 test_blend_x_gbm_mean,
 test_blend_x_gbm_gmean,
 blend_scores_gbm,
 best_rounds_gbm)= lgbm_blend(est, 
                               train_X, train_y, 
                               test_X,
                               10,
                               500) #as the learning rate decreases the number of stopping rounds need to be increased

Blend 5 estimators for 10 folds
Model 1: LGBMClassifier(boosting_type='dart', colsample_bytree=0.296539,
        drop_rate=0.296937, is_unbalance=False, learning_rate=0.01,
        max_bin=347, max_depth=-1, max_drop=50, min_child_samples=105,
        min_child_weight=5, min_split_gain=0, n_estimators=1000000,
        nthread=-1, num_leaves=24, objective='multiclass', reg_alpha=0,
        reg_lambda=0, scale_pos_weight=1, seed=2017, sigmoid=1.0,
        silent=True, skip_drop=0.761489, subsample=0.958292,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)
Model 1 fold 1


KeyboardInterrupt: 

In [5]:

clf = lgb.LGBMClassifier(num_leaves = 24,
                                min_child_samples = 105,
                                colsample_bytree = 0.296539,
                                subsample = 0.958292,
                                max_bin = 347,
                                drop_rate = 0.296937,
                                skip_drop = 0.761489)
clf.set_params(learning_rate = 0.03)
clf.set_params(subsample_freq = 1)
clf.set_params(objective = 'multiclass')
clf.set_params(n_estimators = 100000)
clf.set_params(boosting_type = 'dart')
clf.set_params(seed = seed)

        
clf = clf.fit(X_train, y_train,
              eval_set = [(X_val,y_val)],
              eval_metric = 'multi_logloss',
              early_stopping_rounds = 50,
              verbose = 25)

Train until valid scores didn't improve in 50 rounds.
[25]	valid_0's multi_logloss: 0.865046
[50]	valid_0's multi_logloss: 0.734173
[75]	valid_0's multi_logloss: 0.678042
[100]	valid_0's multi_logloss: 0.643931
[125]	valid_0's multi_logloss: 0.623701
[150]	valid_0's multi_logloss: 0.612349
[175]	valid_0's multi_logloss: 0.597503
[200]	valid_0's multi_logloss: 0.590527
[225]	valid_0's multi_logloss: 0.586382
[250]	valid_0's multi_logloss: 0.581891
[275]	valid_0's multi_logloss: 0.579075
[300]	valid_0's multi_logloss: 0.575144
[325]	valid_0's multi_logloss: 0.572181
[350]	valid_0's multi_logloss: 0.570597
[375]	valid_0's multi_logloss: 0.566742
[400]	valid_0's multi_logloss: 0.564122
[425]	valid_0's multi_logloss: 0.562709
[450]	valid_0's multi_logloss: 0.560168
[475]	valid_0's multi_logloss: 0.558061
[500]	valid_0's multi_logloss: 0.556797
[525]	valid_0's multi_logloss: 0.554282
[550]	valid_0's multi_logloss: 0.553685
[575]	valid_0's multi_logloss: 0.552499
[600]	valid_0's multi_logloss

In [52]:
now = datetime.now()

name_train_blend = '../blend/train_blend_LightGBM_dart_BM_0322_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_mean = '../blend/test_blend_LightGBM_dart_mean_BM_0322_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_gmean = '../blend/test_blend_LightGBM_dart_gmean_BM_0322_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'


print (np.mean(blend_scores_gbm,axis=0))
print (np.mean(best_rounds_gbm,axis=0))
np.savetxt(name_train_blend,train_blend_x_gbm, delimiter=",")
np.savetxt(name_test_blend_mean,test_blend_x_gbm_mean, delimiter=",")
np.savetxt(name_test_blend_gmean,test_blend_x_gbm_gmean, delimiter=",")

[ 0.52309656  0.52262419  0.52356597  0.52188342  0.52198129]
[ 15266.3  12448.   15418.7   7687.9  11589.2]


In [53]:
sub_name = '../output/sub_LightGBM_dart_BM_0322_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'

out_df = pd.DataFrame(test_blend_x_gbm_mean[:,9:12])
out_df.columns = ["low", "medium", "high"]
out_df["listing_id"] = sub_id
out_df.to_csv(sub_name, index=False)